# Create Patch Dataset
This notebook is the source for downloading Sentinel data for the 2D patch classifier

## Inputs
Sites are generated from geojson inputs. The positive TPA sites are defined by polygons, and the negative sites are defined by points. 
New negative sites can be added by following the example of the bootstrap dataset

## Outputs:

### Raw Data:
The output is a dictionary with a structure `[date][site_name][band][band_img]`

### Patches
The output is multispectral patches. `[num_patches, height, width, bands]`

In [1]:
import json
import os
import pickle

import ee
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append('../')
from scripts.get_s2_data_ee import get_history, get_history_polygon, get_pixel_vectors

%load_ext autoreload
%autoreload 2

In [2]:
# Configuration:
# Set directory where training site json files are located and files are saved
# Set rect width for all patches that are not TPA sites
DATA_DIR = '../data/training_sites'
OUTPUT_DIR = '../data/sentinel'
RECT_WIDTH = 0.004
    
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [3]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

In [4]:
def load_points(file_name):
    """Load points saved as a GeoJSON and return a dictionary"""
    with open(os.path.join(DATA_DIR, file_name)) as f:
        sites = json.load(f)
    f.close()

    site_table = pd.DataFrame({
        'name': [file_name.split('_')[0] + '_' + str(index) for index in range(len(sites['features']))],
        'lon': [site['geometry']['coordinates'][0] for site in sites['features']],
        'lat': [site['geometry']['coordinates'][1] for site in sites['features']],
        'coords': [site['geometry']['coordinates'][0:2] for site in sites['features']],
    })
    
    return site_table

## Load Sampling Locations

### Positive Sites

In [16]:
# Positive site coordinates

with open('../data/detection_summary_v12.geojson', 'r') as f:
    positive_sites = json.load(f)['features']
positive_coords = [site['geometry']['coordinates'] for site in positive_sites]
positive_names = [site['properties']['name'] for site in positive_sites]

java_positives = pd.read_csv('../data/training_sites/v12_java_validated_positives.csv', converters={'coords': eval})
for coord in java_positives['coords']:
    positive_coords.append(coord)

for name in list(java_positives['name']):
    positive_names.append(name)

print(len(positive_coords), 'positive sites loaded')

102 positive sites loaded


### Negative Sites

In [6]:
# Create a list of dataframes for default negative site sampling
# Some lists are specifically chosen, and some are generated 
# automatically based on the locations of TPA sites
negative_site_list = [load_points('city_points.geojson'), 
                      load_points('bare_earth_points.geojson'),
                     ]

# New data should likely be added to the bootstrap negatives file
# This will be a part of the negative dataset, but is separate for an example of 
# adding new data to an existing dataset
bootstrap_negatives = load_points('bootstrap_negatives.geojson')

java_negatives = pd.read_csv('../data/training_sites/v12_java_validated_negatives.csv', converters={'coords': eval})

negative_site_list.append(bootstrap_negatives)
negative_site_list.append(java_negatives)

for site in negative_site_list:
    display(site.head())
    



,name,lon,lat,coords
0,city_0,114.619837,-8.361932,"[114.6198374623975, -8.361931821454325]"
1,city_1,115.218992,-8.682543,"[115.2189915773064, -8.682542635447703]"
2,city_2,115.152099,-8.803352,"[115.1520991337562, -8.803351890677076]"
3,city_3,115.448223,-8.676354,"[115.4482234100242, -8.676354239123828]"
4,city_4,115.552125,-8.674258,"[115.5521252514949, -8.674258048038155]"


,name,lon,lat,coords
0,bare_0,115.571182,-8.409224,"[115.5711823425384, -8.409223580393347]"
1,bare_1,115.383874,-8.562584,"[115.3838741922983, -8.562583709771332]"
2,bare_2,115.554872,-8.389197,"[115.5548717252287, -8.389196879431212]"
3,bare_3,115.233932,-8.670693,"[115.2339323116851, -8.670692512141244]"
4,bare_4,115.172205,-8.680163,"[115.1722049239458, -8.68016332592915]"


,name,lon,lat,coords
0,bootstrap_0,115.112336,-8.845362,"[115.112336, -8.845362]"
1,bootstrap_1,115.119432,-8.798492,"[115.119432, -8.798492]"
2,bootstrap_2,115.131650,-8.789526,"[115.13165, -8.789526]"
3,bootstrap_3,115.186896,-8.845007,"[115.186896, -8.845007]"
4,bootstrap_4,115.187076,-8.841368,"[115.187076, -8.841368]"


,name,lon,lat,coords
0,java_0,106.367697,-6.063501,"[106.36769670260679, -6.06350089615559]"
1,java_5,106.814769,-6.722302,"[106.81476931356087, -6.722301883882036]"
2,java_6,106.860191,-6.594631,"[106.86019138413342, -6.594631482332523]"
3,java_7,106.833409,-6.495372,"[106.83340864929384, -6.495372495443919]"
4,java_8,106.833409,-6.494468,"[106.83340864929384, -6.49446767332096]"


In [7]:
negative_data = pd.concat([negative_site_list[0], 
                           negative_site_list[1], 
                           negative_site_list[2],
                           negative_site_list[3]]).reset_index(drop=True)

negative_data.to_csv(os.path.join(DATA_DIR, 'negative_sites.csv'))
print(len(negative_data), 'negative sites')
negative_data.head()

317 negative sites


,name,lon,lat,coords
0,city_0,114.619837,-8.361932,"[114.6198374623975, -8.361931821454325]"
1,city_1,115.218992,-8.682543,"[115.2189915773064, -8.682542635447703]"
2,city_2,115.152099,-8.803352,"[115.1520991337562, -8.803351890677076]"
3,city_3,115.448223,-8.676354,"[115.4482234100242, -8.676354239123828]"
4,city_4,115.552125,-8.674258,"[115.5521252514949, -8.674258048038155]"


## Download Data

### Negative Sites

In [12]:
# Create a list of patch histories
# Each patch history is a dictionary with the format:
# patch_history[date][site_name][band][band_img]
# This function takes a while to run as it is extracting data from GEE
num_negative_months = 6
negative_start_date = '2019-03-01'
negative_patch_histories = get_history(negative_data['coords'], 
                                       negative_data['name'], 
                                       RECT_WIDTH,
                                       num_months=num_negative_months,
                                       start_date=negative_start_date,
                                       cloud_mask=True)

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [13:38<1:08:12, 818.45s/it]

 33%|███▎      | 2/6 [30:20<58:13, 873.48s/it]  

 50%|█████     | 3/6 [46:45<45:20, 906.84s/it]

 67%|██████▋   | 4/6 [1:02:19<30:30, 915.09s/it]

 83%|████████▎ | 5/6 [1:24:57<17:28, 1048.01s/it]

100%|██████████| 6/6 [1:58:22<00:00, 1183.69s/it]


In [13]:
# Save negative patch history
with open(os.path.join(OUTPUT_DIR, f"negative_raw_toa_{RECT_WIDTH}_patch_{num_negative_months}_{negative_start_date}.pkl"),"wb") as f:
    pickle.dump(negative_patch_histories, f)

### Positive Sites

In [18]:
# Get patch histories for TPA sites
num_positive_months = 12
positive_start_date = '2019-01-01'
positive_history = get_history(positive_coords, 
                               positive_names,  
                               RECT_WIDTH,
                               start_date=positive_start_date,
                               num_months=num_positive_months
                              )

  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [21:18<3:54:26, 1278.79s/it]

 17%|█▋        | 2/12 [37:22<3:17:21, 1184.19s/it]

 25%|██▌       | 3/12 [51:46<2:43:14, 1088.31s/it]

 33%|███▎      | 4/12 [1:07:43<2:19:49, 1048.68s/it]

...

 67%|██████▋   | 8/12 [2:11:48<1:04:34, 968.69s/it] 

 75%|███████▌  | 9/12 [2:23:56<44:49, 896.40s/it]  

 83%|████████▎ | 10/12 [2:41:20<31:21, 940.92s/it]

 92%|█████████▏| 11/12 [2:57:11<15:43, 943.71s/it]

100%|██████████| 12/12 [3:07:29<00:00, 937.47s/it]


In [19]:
# Save positive patch history
with open(os.path.join(OUTPUT_DIR, f"positive_raw_toa_{RECT_WIDTH}_patch_{num_positive_months}_{positive_start_date}.pkl"),"wb") as f:
    pickle.dump(positive_history, f)

## Create Spatial Patches

In [20]:
# Load positive patches
with open(os.path.join(OUTPUT_DIR, f"positive_raw_toa_{RECT_WIDTH}_patch_{num_positive_months}_{positive_start_date}.pkl"),"rb") as f:
    positive_patch_histories = pickle.load(f)
    
# Load negative patches
with open(os.path.join(OUTPUT_DIR, f"negative_raw_toa_{RECT_WIDTH}_patch_{num_negative_months}_{negative_start_date}.pkl"),"rb") as f:
    negative_patch_histories = pickle.load(f)

In [21]:
def create_img_stack(patch_history):
    img_stack = []
    for date in patch_history:
        for site in patch_history[date]:
            spectral_stack = []
            band_shapes = [np.shape(patch_history[date][site][band])[0] for band in band_descriptions]
            if np.array(band_shapes).all() > 0:
                for band in band_descriptions:
                    spectral_stack.append(patch_history[date][site][band])
                if np.min(spectral_stack) > 0:
                    img_stack.append(np.rollaxis(np.array(spectral_stack), 0, 3))
    
    min_x = np.min([np.shape(img)[0] for img in img_stack])
    min_y = np.min([np.shape(img)[1] for img in img_stack])
    img_stack = [img[:min_x, :min_y, :] for img in img_stack]
    return img_stack

In [22]:
positive_patches = create_img_stack(positive_patch_histories)
print(len(positive_patches), 'positive images extracted')

negative_patches = create_img_stack(negative_patch_histories)
print(len(negative_patches), 'negative images extracted')

918 positive images extracted
1583 negative images extracted


In [24]:
# Save positive patches
with open(os.path.join(OUTPUT_DIR, f"positive_patches_{RECT_WIDTH}_toa_{num_positive_months}_{positive_start_date}.pkl"),"wb") as f:
    pickle.dump(positive_patches, f)
    
# Save negative patches
with open(os.path.join(OUTPUT_DIR, f"negative_patches_{RECT_WIDTH}_toa_{num_negative_months}_{negative_start_date}.pkl"),"wb") as f:
    pickle.dump(negative_patches, f)